# /imports

In [15]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Read in data

In [16]:
ds = tfds.load('movielens/latest-small-ratings', split='train', shuffle_files=True)

In [17]:
to_str = lambda b: str(b, encoding="utf8")

In [18]:
tinyd = ds.take(1000)
ix = {}
# Should be tinyd.as_numpy_iterator() for convenience
for a in tinyd:
    uid = to_str(a['user_id'].numpy())
    mid = to_str(a['movie_id'].numpy())
    title = a['movie_title']
    print(f"User: {uid} Title: {title} Id: {mid}")
    if uid not in ix:
        ix[uid] = []
    ix[uid].append(mid)

User: 105 Title: b'K-PAX (2001)' Id: 4874
User: 17 Title: b"Schindler's List (1993)" Id: 527
User: 309 Title: b'Killers, The (1946)' Id: 7943
User: 111 Title: b'I Know What You Did Last Summer (1997)' Id: 1644
User: 182 Title: b'Shrek 2 (2004)' Id: 8360
User: 474 Title: b'Ghostbusters II (1989)' Id: 2717
User: 450 Title: b'Shanghai Noon (2000)' Id: 3624
User: 434 Title: b'Heat (1995)' Id: 6
User: 586 Title: b'6th Day, The (2000)' Id: 3986
User: 599 Title: b'Heaven and Earth (Ten to Chi to) (1990)' Id: 57499
User: 362 Title: b'Game, The (1997)' Id: 1625
User: 326 Title: b'Sin City (2005)' Id: 32587
User: 315 Title: b'Children of the Damned (1963)' Id: 2554
User: 182 Title: b'Aliens (1986)' Id: 1200
User: 15 Title: b'Aliens (1986)' Id: 1200
User: 580 Title: b'Spy Game (2001)' Id: 4901
User: 123 Title: b'Training Day (2001)' Id: 4776
User: 219 Title: b'Vertigo (1958)' Id: 903
User: 380 Title: b'Happy Gilmore (1996)' Id: 104
User: 490 Title: b'Mask, The (1994)' Id: 367
User: 560 Title: b'B

User: 352 Title: b'Man in the Iron Mask, The (1998)' Id: 1801
User: 200 Title: b'Shakespeare in Love (1998)' Id: 2396
User: 153 Title: b'Ice Age: Dawn of the Dinosaurs (2009)' Id: 69644
User: 534 Title: b'George of the Jungle (1997)' Id: 1588
User: 41 Title: b'Blood Diamond (2006)' Id: 49530
User: 590 Title: b'Star Wars: Episode VI - Return of the Jedi (1983)' Id: 1210
User: 202 Title: b'Beverly Hills Cop III (1994)' Id: 420
User: 290 Title: b'Iron Giant, The (1999)' Id: 2761
User: 186 Title: b'20 Dates (1998)' Id: 2492
User: 448 Title: b"Big Momma's House (2000)" Id: 3646
User: 168 Title: b"Divorce - Italian Style (Divorzio all'italiana) (1961)" Id: 33564
User: 177 Title: b'Hunchback of Notre Dame, The (1923)' Id: 25746
User: 125 Title: b'The Magnificent Seven (2016)' Id: 162350
User: 432 Title: b'Toy Story 3 (2010)' Id: 78499
User: 599 Title: b'Green Mile, The (1999)' Id: 3147
User: 534 Title: b'I, Robot (2004)' Id: 8644
User: 484 Title: b'Tommy Boy (1995)' Id: 333
User: 160 Title: b

## Filter out users with less than 20 interactions

In [152]:
print(f"Number of users before filter: {len(ix)}")
fix = ix.copy()
for k in list(fix.keys()):
    if len(fix[k]) < 20:
        fix.pop(k)
print(f"Number of users after filter: {len(fix)}")
for k in fix.keys():
    print(f"User {k}, number of ratings: {len(fix[k])}")

Number of users before filter: 342
Number of users after filter: 3
User 599, number of ratings: 27
User 448, number of ratings: 21
User 414, number of ratings: 23


# DAS MODEL

## Parameters

In [135]:
K = 8
MEAN = 0
STD = 0.01

In [146]:
# User ids, Massage
user_ids = [int(x) % 10  for x in fix.keys()]
user_ids

[9, 8, 4]

In [169]:
# Massage items to make them fewer and lower ids
fix2 = {}
for k, v in fix.items():
    fix2[k] = set([int(i) % 70 for i in v])
[print(f, fix2[f]) for f in fix2]

599 {7, 11, 16, 23, 24, 27, 29, 30, 35, 36, 39, 41, 46, 47, 48, 52, 53, 54, 57, 61, 63, 65, 67}
448 {0, 5, 6, 9, 14, 15, 19, 20, 24, 25, 26, 29, 30, 42, 45, 48, 53, 65, 66, 69}
414 {0, 4, 8, 10, 15, 18, 28, 30, 32, 34, 37, 44, 52, 57, 58, 61, 63, 65, 68}


[None, None, None]

In [174]:
# Item ids
item_ids = []
[item_ids.extend(v) for v in fix2.values()]
item_ids = [int(i) for i in item_ids]
max(item_ids)

69

In [207]:
# Constants
M = max(user_ids)+1
N = max(item_ids)+1
print("M:", M, "N:", N)

M: 10 N: 70


In [217]:
user_tensor = tf.one_hot(user_ids, M, axis=0)
user_tensor

<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)>

In [218]:
item_tensor = tf.one_hot(item_ids, N, axis=0)
item_tensor

<tf.Tensor: shape=(70, 62), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [204]:
P

<tf.Tensor: shape=(9, 8), dtype=float32, numpy=
array([[ 0.01395238,  0.00145946, -0.00207921,  0.00351208,  0.02051345,
        -0.00564026, -0.00444468, -0.02514734],
       [ 0.00512499, -0.00970059,  0.00360768, -0.00559165,  0.00456299,
         0.02018761, -0.00654347, -0.01327697],
       [-0.01023071, -0.01530871, -0.01061804, -0.00220184, -0.00739782,
         0.00129344, -0.03654746, -0.00620097],
       [ 0.00478308,  0.00539126,  0.002357  , -0.00105332,  0.0164074 ,
        -0.01560014, -0.00970077, -0.00243381],
       [ 0.00617459, -0.00478502,  0.00301383,  0.02102323,  0.00795626,
         0.00167362, -0.00855466,  0.00677339],
       [-0.01062019,  0.01053117,  0.00744098, -0.00259343, -0.00614429,
         0.00569275, -0.0070251 ,  0.00963446],
       [-0.00489603, -0.00027747,  0.01404346, -0.01606937, -0.01528396,
         0.00776127,  0.01315127,  0.01497171],
       [-0.00622885, -0.00624673, -0.00414677, -0.00768698, -0.00603351,
        -0.00072106, -0.01802851

In [219]:
P = tf.random.normal((M, K), mean=MEAN, stddev=STD, dtype=tf.dtypes.float32)
nex = tf.matmul(tf.transpose(P), user_tensor)
nex

<tf.Tensor: shape=(8, 3), dtype=float32, numpy=
array([[-0.00645911, -0.01529396, -0.00554465],
       [ 0.01140836,  0.00567342, -0.0032734 ],
       [ 0.00123734, -0.01490473,  0.01202611],
       [ 0.01448041,  0.01791899,  0.00713586],
       [-0.00289602, -0.00145756, -0.00445769],
       [-0.00231778, -0.0112207 ,  0.00159613],
       [ 0.00205314,  0.01006553, -0.00174331],
       [-0.00383493, -0.00121818, -0.0044019 ]], dtype=float32)>

# Create test/validation splits

## Test

In [ ]:
# For each user
# # Put aside the latest interaction

## Validation

In [144]:
# For each user
# # Sample random iteraction, tune hyper params on it

# Evaluation

In [ ]:
# For each user
# # Sample 100 items not interacted with
# # Check if test interaction amongst top 10
# # # Hit Rate & NDCG